In [ ]:
# BIBLIOTECAS
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from array import array

from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import gc
import sys
import csv
import copy

In [ ]:
#
def productors_adjustment(panda):
    panda['DATE'] = pd.to_datetime(panda['DATE'], dayfirst=True)
    panda['DATE'] = panda['DATE'].apply(lambda x:x.toordinal())

    # replace null values with 0
    for i in range(panda.shape[1]):
        panda.iloc[:, i] = panda.iloc[:, i].fillna(0)
    
    #panda = panda[['QO', 'QG', 'QW']]
        
def forecast_adjustment(panda):
    panda['DATE'] = pd.to_datetime(panda['DATE'], dayfirst=True, format='mixed')
    panda['DATE'] = panda['DATE'].apply(lambda x:x.toordinal())
    
    # replace null values with 0
    for i in range(panda.shape[1]):
        panda.iloc[:, i] = panda.iloc[:, i].fillna(0)    

In [ ]:
#Productors Data from UEP 01
p_uep01 = []
for i in range(8):
    file_path = "/projetos/bekv/Mestrado/DADOS/HIST/hist_P" + str(i+1) + "_01.csv"
    p_uep01.append(pd.read_csv(file_path, delimiter=";", decimal=","))
    productors_adjustment(p_uep01[i])

#Productors Data from UEP 02
p_uep02 = []
for i in range(9):
    file_path = "/projetos/bekv/Mestrado/DADOS/HIST/hist_P" + str(i+1) + "_02.csv"
    p_uep02.append(pd.read_csv(file_path, delimiter=";", decimal=","))
    productors_adjustment(p_uep02[i])
   

In [ ]:
v_UEP = 1

v_WINDOW = 60
v_BATCH = 128
v_EPOCHS =  25
v_UNITS = 50
v_DROPOUT = 0.0

v_RANDOM = 11032019

v_SCENARIO = "LSTM-NUMS"

GRAPH_TIME = "2020/01"
GRAPH_YEAR = "2020"
GRAPH_MONTH = "01"

if (GRAPH_TIME == "2020/01"):
    SIMULATION_FILE = 0
else:
    if (GRAPH_TIME == "2020/06"):
        SIMULATION_FILE = 1
    else: 
        if (GRAPH_TIME == "2021/01"):
            SIMULATION_FILE = 2
        else:
            if (GRAPH_TIME == "2021/06"):
                SIMULATION_FILE = 3
            else:
                if (GRAPH_TIME == "2022/01"):
                    SIMULATION_FILE = 4
                else:
                    if (GRAPH_TIME == "2022/06"):
                        SIMULATION_FILE = 5
                    else:
                        SIMULATION_FILE = 6

SELECTED_WELL = 1

In [ ]:
#Forecasts for wells and Platforms
forecast_sim = []
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2020_01_daily.csv", delimiter=";", decimal=",", skiprows=[0]))
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2020_06_daily.csv", delimiter=";", decimal=",", skiprows=[0]))
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2021_01_daily.csv", delimiter=";", decimal=",", skiprows=[0]))
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2021_06_daily.csv", delimiter=";", decimal=",", skiprows=[0]))
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2022_01_daily.csv", delimiter=";", decimal=",", skiprows=[0]))
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2022_06_daily.csv", delimiter=";", decimal=",", skiprows=[0]))
forecast_sim.append(pd.read_csv("/projetos/bekv/Mestrado/DADOS/FORECAST/FORECAST_SIM_2022_01_daily.csv", delimiter=";", decimal=",", skiprows=[0])) # not used

In [ ]:
for i in range(7):
    forecast_adjustment(forecast_sim[i])

In [ ]:
date_start = min(p_uep01[0].DATE)
date_start

for i in range(8):
    p_uep01[i]['DATE'] = p_uep01[i]['DATE'] - date_start
    p_uep02[i]['DATE'] = p_uep02[i]['DATE'] - date_start

p_uep02[8]['DATE'] = p_uep02[8]['DATE'] - date_start
    
for i in range(7):
    forecast_sim[i]['DATE'] = forecast_sim[i]['DATE'] - date_start

In [ ]:
filter_hist = (p_uep01[0]['DATE'] < forecast_sim[SIMULATION_FILE]['DATE'][0])

TOTAL_EXTRAPOLATION = 360

filter_extr = ((p_uep01[0]['DATE'] >= forecast_sim[SIMULATION_FILE]['DATE'][0]) & (p_uep01[0]['DATE'] < forecast_sim[SIMULATION_FILE]['DATE'][0] + TOTAL_EXTRAPOLATION))

p_01_training_set = []
p_01_performed_set = []
for i in range(8):
    p_01_training_set.append(p_uep01[i][filter_hist])
    p_01_training_set[i] = p_01_training_set[i].drop('DATE', axis=1)
    p_01_performed_set.append(p_uep01[i][filter_extr])
    p_01_performed_set[i] = p_01_performed_set[i].drop('DATE', axis=1)
    
p_02_training_set = []
p_02_performed_set = []
for i in range(9):
    p_02_training_set.append(p_uep02[i][filter_hist])
    p_02_training_set[i] = p_02_training_set[i].drop('DATE', axis=1)
    p_02_performed_set.append(p_uep02[i][filter_extr])
    p_02_performed_set[i] = p_02_performed_set[i].drop('DATE', axis=1)
    
for i in range(7):
    forecast_sim[i] = forecast_sim[i].drop('DATE', axis=1)

In [ ]:
p_01_training_set_total = p_01_training_set[0][['QO', 'QG', 'QW']]
p_01_performed_set_total = p_01_performed_set[0][['QO', 'QG', 'QW']]
for SELECTED_WELL in range(1,8):
    p_01_training_set_total += p_01_training_set[SELECTED_WELL][['QO', 'QG', 'QW']]
    p_01_performed_set_total += p_01_performed_set[SELECTED_WELL][['QO', 'QG', 'QW']]

p_02_training_set_total = p_02_training_set[0][['QO', 'QG', 'QW']]
p_02_performed_set_total = p_02_performed_set[0][['QO', 'QG', 'QW']]
for SELECTED_WELL in range(1,9):
    p_02_training_set_total += p_02_training_set[SELECTED_WELL][['QO', 'QG', 'QW']]
    p_02_performed_set_total += p_02_performed_set[SELECTED_WELL][['QO', 'QG', 'QW']]
    
forecast_sim_total = forecast_sim[0][['QO_01', 'QG_01', 'QW_01']]
for SIM in range(1,7):
    forecast_sim_total += forecast_sim[SIM][['QO_01', 'QG_01', 'QW_01']]

In [ ]:
scaler = MinMaxScaler()

In [ ]:
p_01_scaled_train_set_total = copy.deepcopy(p_01_training_set_total)
p_02_scaled_train_set_total = copy.deepcopy(p_02_training_set_total)

In [ ]:
#Training Data Normalization

#UEP 01
for feat in range(p_01_scaled_train_set_total.shape[1]):
    scaler.fit(p_01_training_set_total.iloc[:,feat].values.reshape(-1,1))
    p_01_scaled_train_set_total[p_01_scaled_train_set_total.columns[feat]] = scaler.transform(p_01_scaled_train_set_total.iloc[:,feat].values.reshape(-1,1))

#UEP 02
for feat in range(p_02_scaled_train_set_total.shape[1]):
    scaler.fit(p_02_training_set_total.iloc[:,feat].values.reshape(-1,1))
    p_02_scaled_train_set_total[p_02_scaled_train_set_total.columns[feat]] = scaler.transform(p_02_scaled_train_set_total.iloc[:,feat].values.reshape(-1,1))


In [ ]:
if (v_UEP == 1):
    p_01_scaled_train_set_total = p_01_scaled_train_set_total.values
else:
    p_02_scaled_train_set_total = p_02_scaled_train_set_total.values

In [ ]:
def forecast_UEP_01(simulation_file):
    
    forecast = forecast_sim[simulation_file][:TOTAL_EXTRAPOLATION]
    
    QO = forecast.QO_01 / 0.93
    QG = forecast.QG_01
    QW = forecast.QW_01
    
    return QO, QG, QW

In [ ]:
def forecast_UEP_02(simulation_file):
    
    forecast = forecast_sim[simulation_file][:TOTAL_EXTRAPOLATION]
    
    QO = forecast.QO_02 / 0.94
    QG = forecast.QG_02
    QW = forecast.QW_02
    
    return QO, QG, QW

In [ ]:
# Define the function to return the SMAPE value
def smape(actual, predicted) -> float:
    value = 0    
    for i in range(actual.shape[0]):
        value = value + np.abs(actual[i][0] - predicted[i][0]) / ( (np.abs(actual[i][0]) + np.abs(predicted[i][0]) + 0.000001) )

    return (round( 1 * value / actual.shape[0], 3))

def print_smape(name, actual, predicted):
    print(name + " SymetricMeanAbsolutePercentageError (SMAPE):" + str(smape(actual, predicted)))

def file_smape(actual, predicted):
    return str(smape(actual, predicted))

In [ ]:
# Define the function to return the SMAPE value
def mape(actual, predicted) -> float:
    value = 0    
    for i in range(actual.shape[0]):
        value = value + np.abs(actual[i][0] - predicted[i][0]) / ( (np.abs(actual[i][0]) ) + 0.000001)

    return (round( 1 * value / actual.shape[0], 3))

def print_mape(name, actual, predicted):
    print(name + " MeanAbsolutePercentageError (MAPE):" + str(mape(actual, predicted)))

def file_mape(actual, predicted):
    return str(mape(actual, predicted))

In [ ]:
# Define the function to return the SMAPE value
def mae(actual, predicted) -> float:
    value = 0    
    for i in range(actual.shape[0]):
        value = value + np.abs(actual[i][0] - predicted[i][0])

    return (round( 1 * value / actual.shape[0], 3))

def print_mae(name, actual, predicted):
    print(name + " MeanAbsoluteError (MAE):" + str(mae(actual, predicted)))

def file_mae(actual, predicted):
    return str(mae(actual, predicted))

In [ ]:
# Define the function to return the SMAPE value
def mse(actual, predicted) -> float:
    value = 0    
    for i in range(actual.shape[0]):
        value = value + np.abs(actual[i][0] - predicted[i][0]) * np.abs(actual[i][0] - predicted[i][0])

    return (round( 1 * value / actual.shape[0], 3))

def print_mse(name, actual, predicted):
    print(name + " MeanSquaredError (MSE):" + str(mse(actual, predicted)))

def file_mse(actual, predicted):
    return str(mse(actual, predicted))

In [ ]:
#Construção da LSTM
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Dropout
import random as python_random

for v_WINDOW in [30, 60, 120]:
    for v_BATCH in [16, 32, 64, 128]:
        for v_EPOCHS in [25, 50, 100]:
            for v_UNITS in [50, 100]:
                for v_DROPOUT in [0.00, 0.10, 0.30]:
                    
                    print("2020_01;" + str(v_WINDOW) + ";4;" + str(v_BATCH) + ";" + str(v_EPOCHS) + ";" + str(v_UNITS) + ";" + str(v_DROPOUT) + ";TANH;" + "RMSProp")

                    start_time = time.time()
                     
                    TAMANHO_JANELA = v_WINDOW
                    testing_inputs = []
                    y_pred = []
                    
                    X_train = []
                    y_train = []
                    if (v_UEP == 1):
                        for i in range(TAMANHO_JANELA, p_01_scaled_train_set_total.shape[0]):
                            X_train.append(p_01_scaled_train_set_total[i-TAMANHO_JANELA:i, :])
                            y_train.append(p_01_scaled_train_set_total[i, :])
                    else: 
                        for i in range(TAMANHO_JANELA, p_02_scaled_train_set_total.shape[0]):
                            X_train.append(p_02_scaled_train_set_total[i-TAMANHO_JANELA:i, :])
                            y_train.append(p_02_scaled_train_set_total[i, :])
                    
                    X_train, y_train = np.array(X_train), np.array(y_train)
                    X_train.shape, y_train.shape
                    
                    np.random.seed(v_RANDOM) 
                    python_random.seed(v_RANDOM)
                    tf.random.set_seed(v_RANDOM)
                    
                    regressor = Sequential()
                    regressor.add(LSTM(units=v_UNITS, activation = 'tanh', return_sequences = True))
                    regressor.add(Dropout(v_DROPOUT))
                    regressor.add(LSTM(units=v_UNITS, activation = 'tanh', return_sequences = True))
                    regressor.add(Dropout(v_DROPOUT))
                    regressor.add(LSTM(units=v_UNITS, activation = 'tanh', return_sequences = True))
                    regressor.add(Dropout(v_DROPOUT))
                    regressor.add(LSTM(units=v_UNITS, activation = 'tanh'))
                    regressor.add(Dropout(v_DROPOUT))
                    regressor.add(Dense(units=X_train.shape[2]))
                    regressor.compile(optimizer='RMSProp', loss='mean_squared_error') # better choices previously tested
                    
                    history = regressor.fit(X_train, y_train, epochs=v_EPOCHS, batch_size=v_BATCH, shuffle=True, verbose=0)
                        
                    df = []
                    if (v_UEP == 1):
                        df = np.append(p_01_training_set_total[-TAMANHO_JANELA:], np.array(forecast_UEP_01(SIMULATION_FILE)).T, axis = 0)
                        for feat in range(df.shape[1]):
                            scaler.fit(p_01_training_set_total.iloc[:,feat].values.reshape(-1,1))
                            df[:,feat:feat+1] = scaler.transform(df[:,feat].reshape(-1,1))
                    else:
                        df = np.append(p_02_training_set_total[-TAMANHO_JANELA:], np.array(forecast_UEP_02(SIMULATION_FILE)).T, axis = 0)    
                        for feat in range(df.shape[1]):
                            scaler.fit(p_02_training_set_total.iloc[:,feat].values.reshape(-1,1))
                            df[:,feat:feat+1] = scaler.transform(df[:,feat].reshape(-1,1))
                        
                    testing_inputs = df
                        
                    X_test = []
                    y_test = []
                    for i in range(TAMANHO_JANELA, testing_inputs.shape[0]):
                        X_test.append(testing_inputs[i-TAMANHO_JANELA:i])
                        y_test.append(testing_inputs[i, 0])
                    X_test, y_test = np.array(X_test), np.array(y_test)
                    y_pred = regressor.predict(X_test)

                    #Eliminate all negative values to oil, water and gas productions
                    y_pred[:,0][y_pred[:,0] < 0] = 0
                    y_pred[:,1][y_pred[:,1] < 0] = 0
                    y_pred[:,2][y_pred[:,2] < 0] = 0

                    scaler.fit(p_01_training_set_total.iloc[:,0].values.reshape(-1,1))
                    oil_predicted_UEP01 = scaler.inverse_transform(y_pred[:,0].reshape(-1, 1))
                    oil_predicted_UEP01[oil_predicted_UEP01 > 19080] = 19080 # SPU02 processing limit
                    
                    oil_simulated_UEP01 = scaler.inverse_transform(testing_inputs[:,0][-TOTAL_EXTRAPOLATION:].reshape(-1, 1))
                    
                    oil_performed_UEP01 = p_01_performed_set_total.QO.values 
                    
                    scaler.fit(oil_performed_UEP01.reshape(-1,1))

                    string_final = ""
                    string_final = ("2020_01;" + str(v_WINDOW) + ";4;" + str(v_BATCH) + ";" + str(v_EPOCHS) + ";" + 
                                    str(v_UNITS) + ";" + str(v_DROPOUT) + ";TANH;" + "RMSProp;" +                                  
                                    file_smape(scaler.transform(oil_predicted_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_smape(scaler.transform(oil_simulated_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_mae(scaler.transform(oil_predicted_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_mae(scaler.transform(oil_simulated_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_mape(scaler.transform(oil_predicted_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_mape(scaler.transform(oil_simulated_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_mse(scaler.transform(oil_predicted_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))) + ";" + 
                                    file_mse(scaler.transform(oil_simulated_UEP01.reshape(-1,1)), scaler.transform(oil_performed_UEP01.reshape(-1,1))))
                    
                    string_final = (string_final + ";" + 
                                    str(round(np.mean(oil_performed_UEP01),2)) + ";" +
                                    str(round(np.mean(oil_predicted_UEP01),2)) + ";" +
                                    "0;" + # to be filled in later with the difference between the productions in the spreadsheet
                                    str(round(np.mean(oil_simulated_UEP01),2)) + ";" +
                                    "0") # to be filled in later with the difference between the productions in the spreadsheet

                    scaler.fit(p_01_training_set_total.iloc[:,2].values.reshape(-1,1))
    
                    wat_predicted_UEP01 = scaler.inverse_transform(y_pred[:,2].reshape(-1, 1))           
                    wat_simulated_UEP01 = scaler.inverse_transform(testing_inputs[:,2][-TOTAL_EXTRAPOLATION:].reshape(-1, 1))          
                    wat_performed_UEP01 = p_01_performed_set_total.QW.values
                    
                    scaler.fit(wat_performed_UEP01.reshape(-1,1))
                    string_final = (string_final + ";" +
                                    file_smape(scaler.transform(wat_predicted_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_smape(scaler.transform(wat_simulated_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_mae(scaler.transform(wat_predicted_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_mae(scaler.transform(wat_simulated_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_mape(scaler.transform(wat_predicted_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_mape(scaler.transform(wat_simulated_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_mse(scaler.transform(wat_predicted_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    file_mse(scaler.transform(wat_simulated_UEP01.reshape(-1,1)), scaler.transform(wat_performed_UEP01.reshape(-1,1))) + ";" +
                                    str(round(np.mean(wat_performed_UEP01),2)) + ";" +
                                    str(round(np.mean(wat_predicted_UEP01),2)) + ";" +
                                    "0;" + 
                                    str(round(np.mean(wat_simulated_UEP01),2)) + ";" +
                                    "0;" + str((time.time() - start_time)/60))

                    with open("_LSTM_NUMS_VALIDATION_RESULTS.txt", "a") as file:
                        file.write(string_final + "\n") 
                        print(string_final)
                    